# Сентимент моделирование

В предыдущих сериях:

1) Сделали обычный процессинг - столбец `cleaned_text`
2) Сделали лемматизацию - `lemmatized_text`
3) По lemmatized_text сделали селекцию от 6 слов и больше


In [1]:
import numpy as np

# 1
import pandas as pd

import yaml
import pathlib

ROOT = pathlib.Path().resolve().parent.parent

with open(ROOT / "config.yml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

DATA_PATH = ROOT / config['api']['DATA_PATH']

In [2]:
# 2

dataset = pd.read_csv(DATA_PATH / 'comment' / 'cm_dataset.csv')
dataset

,index,id,snippet.videoId,snippet.textDisplay,snippet.textOriginal,snippet.authorDisplayName,snippet.parentId,cleaned_text,lemmatized_text,Разметка даннных
0,27894,UgxhFoxSqx1yAYT7atp4AaABAg,Li9V9D0b_hM,Ксюша спасибо за ваш труд и за полезную информ...,Ксюша спасибо за ваш труд и за полезную информ...,Natalja Sk,-1,Ксюша спасибо за ваш труд и за полезную информ...,ксюша спасибо ваш труд полезный информация дум...,NaN
1,20820,UgwTTZmqC3dcnrL3f5B4AaABAg,HLNcjqIq7gs,Макияж сегодня особенно красивый;) пока пробов...,Макияж сегодня особенно красивый;) пока пробов...,Diana Suvorova,-1,Макияж сегодня особенно красивый;) пока пробов...,макияж сегодня особенно красивый пока пробоват...,NaN
2,10202,UgySgzy2iZMjLLcDxhh4AaABAg,j1IdDdQcJng,"Никогда не смотрела Лиссу, но сейчас уже 3 ден...","Никогда не смотрела Лиссу, но сейчас уже 3 ден...",Призрак,-1,"Никогда не смотрела Лиссу, но сейчас уже 3 ден...",смотреть лисс день смотреть уютный веселый видео,NaN
3,16470,UgwVuVcwFxZwVSPmC3Z4AaABAg,IkzWyECzlco,Руки Кёнха столько всего повидали за несколько...,Руки Кёнха столько всего повидали за несколько...,Zoro Roronoa,-1,Руки Кёнха столько всего повидали за несколько...,рука кенх столько повидать несколько минута,NaN
4,29970,Ugw029ioXJAgbOWrCA94AaABAg.9cwNxc_Z_4M9dM4AHOpjsE,Wi49rKRM85U,А ретиноид как вы используете? Всё хочу начать...,А ретиноид как вы используете? Всё хочу начать...,brainbrain,Ugw029ioXJAgbOWrCA94AaABAg,А ретиноид как вы используете? Всё хочу начать...,ретиноид использовать хотеть начинать знать по...,NaN
...,...,...,...,...,...,...,...,...,...,...
5447,41765,UgyNMcGbdzS3PJkhNp14AaABAg.9kO9yJDDrCh9kOOwLQ1tsf,boB2LEXG_zA,@mila ya я как то помыла мылом лицо месяца два...,@mila ya я как то помыла мылом лицо месяца два...,Марина Марина,NaN,ya я как то помыла мылом лицо месяца два.. не...,ya помыть мыло лицо месяц спец средство мягко ...,0.0
5448,41769,UgyNMcGbdzS3PJkhNp14AaABAg.9kO9yJDDrCh9kONm9pHXJ_,boB2LEXG_zA,"@Natali N во первых, я всю жизнь проработала в...","@Natali N во первых, я всю жизнь проработала в...",mila ya,NaN,"N во первых, я всю жизнь проработала в кабине...",n первый весь жизнь прорабатывать кабинет пыль...,2.0
5449,41771,UgyNMcGbdzS3PJkhNp14AaABAg.9kO9yJDDrCh9kOLqLSTzR5,boB2LEXG_zA,"@ЗаноZa , Да-да, продажная цена три копейки, а...","@ЗаноZa , Да-да, продажная цена три копейки, а...",Natali N,NaN,"@ЗаноZa , Да-да, продажная цена три копейки, а...",заноza продажный цена копейка закупочный минус...,2.0
5450,41777,UgyNMcGbdzS3PJkhNp14AaABAg.9kO9yJDDrCh9kOCWb6ImX5,boB2LEXG_zA,"А мне она не подходит , на лице скатывается . ...","А мне она не подходит , на лице скатывается . ...",черная мамба,NaN,"А мне она не подходит , на лице скатывается . ...",подходить лицо скатываться просто нивея мазюка...,0.0


In [3]:
# !pip install transformers sentencepiece --quiet
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny2-cedr-emotion-detection'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
    model.cuda()



In [4]:
def get_sentiment(text, return_type='label'):
    """ Calculate sentiment of a text. `return_type` can be 'label', 'score' or 'proba' """
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
    if return_type == 'label':
        return model.config.id2label[proba.argmax()]
    elif return_type == 'score':
        return proba.dot([1, 1, 1, 1, 1, 1])
    return proba

test_text = dataset['cleaned_text'][0]
print(test_text)
    # classify the text
print(get_sentiment(test_text, 'label'))  # negative
    # score the text on the scale from -1 (very negative) to +1 (very positive)
print(get_sentiment(test_text, 'score'))  # -0.5894946306943893
    # calculate probabilities of all labels
print(get_sentiment(test_text, 'proba'))  # [0.7870447  0.4947824  0.19755007]

Ксюша спасибо за ваш труд и за полезную информацию .😊Я думаю  Вебинар отличная   идея👍🏻😊
joy
0.9943837854079902
[0.3408624  0.63062835 0.00296949 0.01297583 0.00276205 0.00418568]


In [5]:
from tqdm import tqdm

sentiments = []

for id, text in tqdm(zip(dataset['index'], dataset['cleaned_text'])):

    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
        label = model.config.id2label[proba.argmax()]
        score = proba.dot([1, 1, 1, 1, 1, 1])

    sentiments.append({'index': id, 'label': label, 'score': np.max(score)})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


5452it [00:28, 190.55it/s]


In [10]:
sent_df = pd.DataFrame.from_records(sentiments)

display(sent_df.sample(10))

,index,label,score
3642,20741,joy,1.041032
3838,32336,joy,1.000060
3122,2726,joy,1.011882
1392,11863,sadness,1.050631
5326,28654,joy,1.077481
4797,30318,joy,1.038943
45,35327,joy,1.043720
2851,861,no_emotion,1.018172
2059,34837,joy,0.974486
2399,28642,no_emotion,1.015934


In [11]:
sent_df['text'] = dataset['cleaned_text']
sent_df['comment_id'] = dataset['id']
sent_df['snippet.videoId'] = dataset['snippet.videoId']


In [13]:
sent_df.sample(10)

,index,label,score,text,comment_id,snippet.videoId
1299,30387,joy,0.994815,".Zhu , здрасьте, у них референдум по изменению...",UgzjilKhMywWy0oGID14AaABAg.9Yx47jtqbKp9YxOmBQ_Zj9,SzEUlOe0gkU
5180,245,sadness,1.045079,Я так купила фэйковый крем Академия за 50 евро...,Ugz-7c6UNXL_00wY4j54AaABAg,AAjsSrjZXsw
385,35681,joy,1.009018,Здравствуйте Сауле. Спасибо большое за Ваше вн...,UgxemMoLSCT3_VDgc5N4AaABAg.9BrqQPBrmVI9BtOa9xtfgr,LiW9s4YrMFk
1352,37280,no_emotion,1.015874,ТЭЦ это другое. В Южную Корею (точнее вообще ...,Ugy-CJ7dtaRW632ZBZp4AaABAg.9STtvEPlckx9VRIaA7pYwu,2XPnul7asr0
527,7122,surprise,1.138613,"Кёнха, а разве в Корее бывают невежливые прода...",UgzOF8dvjmnvWmj8_Tx4AaABAg,_F2MLvKc1Zo
645,11713,no_emotion,1.002340,Спасибо за позновательный контент. Скажите вы ...,Ugw73D9wEA4xVmCcApp4AaABAg,w9E74hDKVuQ
1588,3885,no_emotion,1.015201,Darling вроде как бренд самого золотого яблока...,UgzN1iiV8UG8HOxqqsx4AaABAg,4nqfUzIz7kQ
2457,33704,joy,1.023280,Ольга Солнечная на губах белордизайн million k...,UgzFLmjJTU5ySG_5-hF4AaABAg.94wljQ8emiL94wp2-WQAxd,lkHQzaSBnFw
1116,13611,joy,1.042304,Доброе утро! Спасибо за видео😀 Волосы - шикарны👍,UgzTV_H2Fs56bp6vAD54AaABAg,WtKIVeULhMs
4593,20948,joy,1.076311,"Ооочень ждала обзор Purito,спасибо,Ксюша. Ещё ...",Ugwn0x2pxHjPUQ7KxTN4AaABAg,HLNcjqIq7gs


In [14]:
sent_df.to_excel(DATA_PATH / 'comment/cm_sentiment.xlsx')